In [2]:
import pandas as pd
import glob
import math
import numpy as np

from helpers import *

v_data='v2'
v_model=['m05i{}_6f'.format(i) for i in range(5)]

n_files=2

In [3]:
v_model

['m05i0_6f', 'm05i1_6f', 'm05i2_6f', 'm05i3_6f', 'm05i4_6f']

# Get the validation files

In [4]:
data_path = 'data/' # TODO: point this to your data folder
#submission_path = data_path + 'submissions/'
training_path = data_path + 'training_set/'
#fin_X = sorted(glob.glob(training_path + "X_val*.csv.gz")) # TODO: point this to a subset of csv's in your training set
#fin_y = sorted(glob.glob(training_path + "y_val*.csv.gz"))
fins = sorted(glob.glob(training_path + "outDD_"+v_data+"_*.h5"))
trn_extra = sorted(glob.glob(training_path + "outDD_v4_*.h5"))
ys   = sorted(glob.glob(training_path + "y_*_.h5"))

In [5]:
fins

['data/training_set/outDD_v2_00.h5',
 'data/training_set/outDD_v2_01.h5',
 'data/training_set/outDD_v2_02.h5',
 'data/training_set/outDD_v2_03.h5',
 'data/training_set/outDD_v2_04.h5',
 'data/training_set/outDD_v2_05.h5',
 'data/training_set/outDD_v2_06.h5',
 'data/training_set/outDD_v2_07.h5',
 'data/training_set/outDD_v2_08.h5',
 'data/training_set/outDD_v2_09.h5',
 'data/training_set/outDD_v2_10.h5',
 'data/training_set/outDD_v2_11.h5',
 'data/training_set/outDD_v2_12.h5',
 'data/training_set/outDD_v2_13.h5',
 'data/training_set/outDD_v2_14.h5',
 'data/training_set/outDD_v2_15.h5',
 'data/training_set/outDD_v2_16.h5',
 'data/training_set/outDD_v2_17.h5',
 'data/training_set/outDD_v2_18.h5',
 'data/training_set/outDD_v2_19.h5',
 'data/training_set/outDD_v2_20.h5',
 'data/training_set/outDD_v2_21.h5',
 'data/training_set/outDD_v2_22.h5',
 'data/training_set/outDD_v2_23.h5',
 'data/training_set/outDD_v2_24.h5',
 'data/training_set/outDD_v2_25.h5',
 'data/training_set/outDD_v2_26.h5',
 

In [6]:
ys

['data/training_set/y_00_.h5',
 'data/training_set/y_01_.h5',
 'data/training_set/y_02_.h5',
 'data/training_set/y_03_.h5',
 'data/training_set/y_04_.h5',
 'data/training_set/y_05_.h5',
 'data/training_set/y_06_.h5',
 'data/training_set/y_07_.h5',
 'data/training_set/y_08_.h5',
 'data/training_set/y_09_.h5',
 'data/training_set/y_10_.h5',
 'data/training_set/y_11_.h5',
 'data/training_set/y_12_.h5',
 'data/training_set/y_13_.h5',
 'data/training_set/y_14_.h5',
 'data/training_set/y_15_.h5',
 'data/training_set/y_16_.h5',
 'data/training_set/y_17_.h5',
 'data/training_set/y_18_.h5',
 'data/training_set/y_19_.h5',
 'data/training_set/y_20_.h5',
 'data/training_set/y_21_.h5',
 'data/training_set/y_22_.h5',
 'data/training_set/y_23_.h5',
 'data/training_set/y_24_.h5',
 'data/training_set/y_25_.h5',
 'data/training_set/y_26_.h5',
 'data/training_set/y_27_.h5',
 'data/training_set/y_28_.h5',
 'data/training_set/y_29_.h5',
 'data/training_set/y_30_.h5',
 'data/training_set/y_31_.h5',
 'data/t

In [7]:
trn_extra

['data/training_set/outDD_v4_00.h5',
 'data/training_set/outDD_v4_01.h5',
 'data/training_set/outDD_v4_02.h5',
 'data/training_set/outDD_v4_03.h5',
 'data/training_set/outDD_v4_04.h5',
 'data/training_set/outDD_v4_05.h5',
 'data/training_set/outDD_v4_06.h5',
 'data/training_set/outDD_v4_07.h5',
 'data/training_set/outDD_v4_08.h5',
 'data/training_set/outDD_v4_09.h5',
 'data/training_set/outDD_v4_10.h5',
 'data/training_set/outDD_v4_11.h5',
 'data/training_set/outDD_v4_12.h5',
 'data/training_set/outDD_v4_13.h5',
 'data/training_set/outDD_v4_14.h5',
 'data/training_set/outDD_v4_15.h5',
 'data/training_set/outDD_v4_16.h5',
 'data/training_set/outDD_v4_17.h5',
 'data/training_set/outDD_v4_18.h5',
 'data/training_set/outDD_v4_19.h5',
 'data/training_set/outDD_v4_20.h5',
 'data/training_set/outDD_v4_21.h5',
 'data/training_set/outDD_v4_22.h5',
 'data/training_set/outDD_v4_23.h5',
 'data/training_set/outDD_v4_24.h5',
 'data/training_set/outDD_v4_25.h5',
 'data/training_set/outDD_v4_26.h5',
 

# Read in the data

In [16]:
start_file = 50
df_trn = pd.concat([read_log(f) for f in fins[start_file:start_file+n_files]], axis=0, ignore_index=True)
df_trn.rename({'not_skipped': 'skip_4'}, axis=1, inplace=True)
df_xtr = pd.concat([read_log(f) for f in trn_extra[start_file:start_file+n_files]], axis=0, ignore_index=True)
df_trn = pd.concat([df_trn, df_xtr], axis=1)

from sklearn.preprocessing import LabelEncoder
df_trn['session_id'] = LabelEncoder().fit_transform(df_trn['session_id'])
df_trn['session_id'] = df_trn['session_id'].astype(np.uint32)
gc.collect()

n_trk=10
X_trn, y_trn, X_trk = get_XY(df_trn, aggs, reset_index=False, 
                  list_musik_qualities_=list_musik_qualities,
                  aggs_music_qualities_=aggs_music_qualities,
                   i_=list(range(n_trk)),
                            aggs_trkvec_=aggs_trkvec, list_trkvec_=list_trkvec)

del df_trn
gc.collect()

756

# Get the ground truth

In [9]:
#ground_truth = pd.Series(get_y_truth(df_trn))
ground_truth = pd.concat([pd.read_hdf(f, key='df') for f in ys[start_file:start_file+n_files]], axis=0)

In [10]:
ys[start_file:start_file+n_files]

['data/training_set/y_50_.h5', 'data/training_set/y_51_.h5']

In [11]:
ground_truth.head()

0                   [1, 1, 1, 0, 0]
1             [1, 1, 0, 1, 1, 1, 1]
2    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
3    [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
4       [1, 1, 1, 0, 0, 0, 1, 1, 1]
dtype: object

# Make predictions using pre-trained models

In [17]:
l_prob = []
for i_ in tqdm(list(range(n_trk))):
    X = pd.concat([X_trn, X_trk[i_]], axis=1)
    if len(l_prob) > 0:
        X = pd.concat([X, pd.DataFrame({'pred_trk{}'.format(j): l_prob[j] 
                                        for j in range(len(l_prob))
                                       })],
                      axis=1)
#         display(X.head())
    import joblib
    prob_pred = np.zeros(shape=(len(X),))
    for v in v_model:
        mfn = 'models/model_{}_{}_{}.pkl'.format(v_data, v, i_)
#         print(mfn)
        mdl = joblib.load(mfn)
        prob_pred += mdl.predict_proba(X)[:,1]

    prob_pred /= len(v_model)
    l_prob.append(prob_pred)

# transform accumulated predicted probabilities into competition predictions           
preds = pred_series_of_lists([p>0.50 for p in l_prob], ground_truth.apply(len), i_2fill=-2)

100%|██████████| 10/10 [01:10<00:00,  6.16s/it]


In [18]:
preds.head(20)

0                    [1, 1, 0, 0, 0]
1              [1, 1, 1, 1, 1, 0, 0]
2     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
3     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4        [0, 1, 1, 0, 0, 0, 0, 0, 0]
5     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
6                 [1, 1, 0, 0, 0, 0]
7           [1, 1, 1, 1, 0, 0, 0, 0]
8     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
9                 [1, 0, 0, 0, 0, 0]
10       [1, 1, 1, 1, 1, 1, 1, 1, 0]
11    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
12    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
13    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
14    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
15    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
16    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
17                [1, 1, 1, 1, 0, 0]
18    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
19          [0, 0, 0, 0, 0, 0, 0, 0]
dtype: object

# Evaluate the predictions

In [19]:
ap,first_pred_acc = evaluate(preds.tolist(), ground_truth.tolist())

In [20]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.5799379348158507
first prediction accuracy: 0.7804511487842962


# Do recursive evaluation

In [20]:
for i in range(1,n_trk+1):
    print('------------ {} ----------------'.format(i))
    preds = pred_series_of_lists([l_prob[i_]>0.50 for i_ in range(i)], ground_truth.apply(len), i_2fill=-2)
    ap,first_pred_acc = evaluate(preds, ground_truth)
    print('average precision: {:.4f}'.format(ap))
#     print('first prediction accuracy: {}'.format(first_pred_acc))

------------ 1 ----------------
average precision: 0.5539
------------ 2 ----------------
average precision: 0.5613
------------ 3 ----------------
average precision: 0.5640
------------ 4 ----------------
average precision: 0.5672
------------ 5 ----------------
average precision: 0.5703
